In [ ]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='30GB')
client

In [ ]:
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import scipy as sp
import pandas as pd
from netCDF4 import Dataset
import dask
import torch
import torch.nn as nn
import torch.fft as fft
import subprocess
import sys

from sht_utils import *
from subs1_utils import *

In [ ]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 42 # zonal wave number
mw = 42 # meridional wave number
kmax = 11
imax = 128
jmax = 64
steps_per_day = 216 ### Changing this number impliws time step changes and should
#                       be implemented carefully
###
spec = (mw,zw,kmax)
grid = (imax,jmax,kmax)
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'Control'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
subprocess.call(['rm','-r', datapath])
subprocess.check_output(['mkdir', datapath])

In [ ]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg = legendre_gauss_weights(jmax, -1, 1)
lats = np.flip(np.arccos(cost_lg))
lats = -90+180*lats/(np.pi)
#
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Initialize grid to spectral (sht) and spectral to grid (isht)
# transforms
#
sht = RealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
isht = InverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
vsht = RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
ivsht = InverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [ ]:
#
#
#
# Initialize spectral fields (at rest or to be read in)
def initialize(f_spec,temp_newton,lnpsclim,kmax,mw,zw):
    for k in range (kmax):
        zmn1[k] = zmn1[k]+f_spec
        tmn1[k] = tmn1[k] + temp_newton[k]
        zmn2[k] = zmn2[k]+f_spec
        tmn2[k] = tmn2[k] + temp_newton[k]
        zmn3[k] = zmn3[k]+f_spec
        tmn3[k] = tmn3[k] + temp_newton[k]
    qmn1 = lnpsclim
    qmn2 = lnpsclim
    qmn3 = lnpsclim
    return zmn1,zmn2,zmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3

In [ ]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook gptosp.agcm.ipynb
#              
#
# Implement at rest initial condition, but need coriolis since
# model predicts total vorticity
#
coriolis = np.zeros((jmax,imax))
for jj in range(jmax):
    coriolis[jj,:] = (4.0*np.pi/86400)*np.sin(-lats[jj]*np.pi/180.0)
#
f_spec = sht(torch.from_numpy(coriolis)) # f_spec is the spectral 
                                         # coriolis parameter
#
# Initialize spectral fields (at rest or to be read in)
# Need to read in background temperature data for Newtonian
# Relaxation and possible initialization, see gptosp.agcm.ipynb
# for how to change source data or formulation.
#
temp_newton = torch.load('temp.spectral.pt') # newtonian 
#                                    relaxation temperature, spectral
lnpsclim = torch.load('lnps.spectral.pt') # lnps climatology for damping
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
zmn1,zmn2,zmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 =\
initialize(f_spec,temp_newton,lnpsclim,kmax,mw,zw)
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see gptosp.agcm.ipynb for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
###phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
# If non-zero topog read here
#
phismn = torch.load('topog.spectral.pt')
#
#
# Adding heating here see preprocess.ipynb
#
heat = torch.load('heat.ggrid.pt')
#
# or set to zero
#
###heat = torch.zeros((kmax,jmax,imax),dtype=torch.complex128)
#

In [ ]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [ ]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
junk = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ttend = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
temp = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
qdot = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 12
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        zmnt[daycount] = zmn1
        dmnt[daycount] = dmn1
        tmnt[daycount] = tmn1
        qmnt[daycount] = qmn1
        wmnt[daycount] = wmn1
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl:total_days]
            postprocessing(isht,ivsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,f_spec,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            daycount = 0
        savedat = 0
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    for k in range(kmax):
        vort[k] = isht(zmn2[k]).cpu() ### This is the absolute vorticity
        div[k] = isht(dmn2[k]).cpu()
        temp[k] = isht(tmn2[k]).cpu()
        qdot[k] = isht(wmn2[k]).cpu()
    u,v = uv(ivsht,zmn2,dmn2,f_spec,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(ivsht,qmn2,mw,zw,kmax,imax,jmax)
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod(u,v,vort,div,temp,dxq,dyq,heat,delsig,si,sikap,slkap,\
                                         r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv
    junk,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    for k in range(kmax):
        dddt = dmn3[k] - lap_sht(sht,e[k],mw,zw) 
        dmn3[k] = dddt
        tmn3[k] = -ttend[k] + sht(ri[k]).cpu()
        wmn3[k] = sht(wj[k]).cpu() ### Prescribed heating converted to spectral
    qmn3 = -sht(cbar).cpu() ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,\
                            kmax,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,\
                          f_spec,temp_newton,lnpsclim,kmax)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done